In [1]:
import json
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import openweathermapy.core as owm
#import os
#import csv
import seaborn as sns

import gmaps
import gmaps.geojson_geometries

from Project_Gas_Library import f_EIA_ListSeries, f_EIA_ExtractData, f_PlotData, f_EIA_PlotData, f_EIA_PlotData_Mod,\
f_WorldWeather_ExtractData, f_PlotData, f_Census_ExtractData, calculate_color, f_strToDate, f_EIA_ExtractData_Range, \
f_EIA_PlotData_Range, f_EIA_PlotData_Mod_Range, f_EIA_PlotData_Stack_Range, adf, RMSE, calculate_color_1

from datetime import datetime
from dateutil.relativedelta import relativedelta
from calendar import monthrange

from config import worldweather_key, g_key, EIA_key, census_key

from matplotlib.pyplot import cm
from citipy import citipy
from pprint import pprint

from matplotlib.cm import viridis
from matplotlib.colors import to_hex

from census import Census

#from matplotlib.cm import pink

In [23]:
ProdYear = 2011

In [24]:
file = "county_production.csv"
df = pd.read_csv(file)
df = df[["State","County_Name","Year","Mscf"]]
df["County_Name"]=df['County_Name'].str.replace(' County','',regex=True)
df_county_year=df.loc[df["Year"]==ProdYear,["State","County_Name","Mscf"]]
#df_county_year

In [4]:
gmaps.configure(api_key=g_key)
gmap_geo_us_counties = gmaps.geojson_geometries.load_geometry('us-counties')

fig = gmaps.figure()

gmap_geo_layer = gmaps.geojson_layer(gmap_geo_us_counties)
fig.add_layer(gmap_geo_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [25]:
vx = list(df_county_year["County_Name"])
vy  = list(df_county_year["Mscf"])
dict_par = dict(zip(vx,vy))

min_par = np.power(min(list(df["Mscf"])),0.10)
max_par = np.power(max(list(df["Mscf"])),0.10)

#min_par = min(vy)
#max_par = max(vy)

colors = []
for feature in gmap_geo_us_counties['features']:
    county_name = feature['properties']['NAME']
    try:
        par = np.power(dict_par[county_name],0.10)
        #par = dict_par[county_name]
        color = calculate_color_1(par,min_par,max_par)
    except KeyError:
        # no value for that county: return default color
        color = (0, 0, 0, 0.3)
    colors.append(color)

fig = gmaps.figure()
poverty_layer = gmaps.geojson_layer(gmap_geo_us_counties,fill_color=colors,stroke_color=colors,fill_opacity=0.8)
#gini_layer = gmaps.geojson_layer(x,fill_color=colors,stroke_color=colors)
fig.add_layer(poverty_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [ ]:
file = "Data/cleandata/dpr_region_counties.csv"
df_region_counties = pd.read_csv(file)
df.head()

file = "Data/cleandata/dpr_anadarko.csv"
df_anadarko = pd.read_csv(file)
df_anadarko = df_anadarko.rename(columns={'Month': 'Date'})
df_anadarko['Date'] = pd.to_datetime(df_anadarko['Date'])

file = "Data/cleandata/dpr_appalachia.csv"
df_appalachia = pd.read_csv(file)
df_appalachia = df_appalachia.rename(columns={'Month': 'Date'})
df_appalachia['Date'] = pd.to_datetime(df_appalachia['Date'])

file = "Data/cleandata/dpr_bakken.csv"
df_bakken = pd.read_csv(file)
df_bakken = df_bakken.rename(columns={'Month': 'Date'})
df_bakken['Date'] = pd.to_datetime(df_bakken['Date'])

file = "Data/cleandata/dpr_eagle_ford.csv"
df_eagle_ford = pd.read_csv(file)
df_eagle_ford = df_eagle_ford.rename(columns={'Month': 'Date'})
df_eagle_ford['Date'] = pd.to_datetime(df_eagle_ford['Date'])

file = "Data/cleandata/dpr_haynesville.csv"
df_haynesville = pd.read_csv(file)
df_haynesville = df_haynesville.rename(columns={'Month': 'Date'})
df_haynesville['Date'] = pd.to_datetime(df_haynesville['Date'])

file = "Data/cleandata/dpr_niobrara.csv"
df_niobrara = pd.read_csv(file)
df_niobrara = df_niobrara.rename(columns={'Month': 'Date'})
df_niobrara['Date'] = pd.to_datetime(df_niobrara['Date'])

file = "Data/cleandata/dpr_permian.csv"
df_permian = pd.read_csv(file)
df_permian = df_permian.rename(columns={'Month': 'Date'})
df_permian['Date'] = pd.to_datetime(df_permian['Date'])

In [ ]:
df_permian.head()

In [ ]:
df_region_counties.head()

In [ ]:
Year = 2012
Month = 2

prod =df_permian.loc[df_permian["Date"]==datetime(Year,Month,1)]["Total production.1"].values[0]

In [ ]:
lcounties = df_region_counties.loc[df_region_counties["Region"]=="Permian Region"]["County"].tolist()

elements = len(lcounties)
number = prod/elements

lprod = [number] * elements

avg_prod_max = 0
avg_prod_min = 0

for j in range(2012,2019,1):
    Year = j
    Month = 6    
    for i in range(0,7,1):        
        if i ==0:
            df = df_anadarko
            str_search = "Anadarko Region"
        elif i==1:
            df = df_appalachia
            str_search = "Appalachia Region"
        elif i==2:
            df = df_bakken
            str_search = "Bakken Region"
        elif i==3:
            df = df_eagle_ford
            str_search = "Eagle Ford Region"
        elif i==4:
            df = df_haynesville
            str_search = "Haynesville Region"
        elif i==5:
            df = df_niobrara
            str_search = "Niobrara Region"
        elif i==6:
            df = df_permian
            str_search = "Permian Region"
            
        prod = df.loc[df["Date"]==datetime(Year,Month,1)]["Total production.1"].values[0]
        lcounties = df_region_counties.loc[df_region_counties["Region"]==str_search]["County"].tolist() 

        NumCounties = len(lcounties)
        avg_prod = prod/NumCounties

        if (j == 2012) & (i==0) :
            avg_prod_max = avg_prod
            avg_prod_min = avg_prod
        else:            
            if avg_prod_max < avg_prod:
                avg_prod_max = avg_prod
            
            if avg_prod_min > avg_prod:
                avg_prod_min = avg_prod

In [ ]:
Year = 2018
Month = 6

list_counties = []
list_prod = []
avg_prod_max = 0

for i in range(0,7,1):
    if i ==0:
        df = df_anadarko
        str_search = "Anadarko Region"
    elif i==1:
        df = df_appalachia
        str_search = "Appalachia Region"
    elif i==2:
        df = df_bakken
        str_search = "Bakken Region"
    elif i==3:
        df = df_eagle_ford
        str_search = "Eagle Ford Region"
    elif i==4:
        df = df_haynesville
        str_search = "Haynesville Region"
    elif i==5:
        df = df_niobrara
        str_search = "Niobrara Region"
    elif i==6:
        df = df_permian
        str_search = "Permian Region"
        
    prod =df.loc[df["Date"]==datetime(Year,Month,1)]["Total production.1"].values[0]
    lcounties = df_region_counties.loc[df_region_counties["Region"]==str_search]["County"].tolist()
    
    NumCounties = len(lcounties)
    avg_prod = prod/NumCounties

    lprod = [avg_prod] * NumCounties
    
    list_counties.extend(lcounties)
    list_prod.extend(lprod)


dict_par = dict(zip(list_counties,list_prod))

#min_par = np.power(avg_prod_min,0.5)
#max_par = np.power(avg_prod_max,0.5)

min_par = np.exp(avg_prod_min)
max_par = np.exp(avg_prod_max)

#min_par = min(vy)
#max_par = max(vy)

colors = []
for feature in gmap_geo_us_counties['features']:
    county_name = feature['properties']['NAME']
    county_name = county_name.upper()
    try:
        #par = np.power(dict_par[county_name],0.5)
        par = np.exp(dict_par[county_name])
        #par = dict_par[county_name]
        color = calculate_color_1(par,min_par,max_par)
    except KeyError:
        # no value for that county: return default color
        #color = (0, 0, 0, 0.3)
        color = (0, 0, 0, 0.0)
    colors.append(color)

fig = gmaps.figure()
poverty_layer = gmaps.geojson_layer(gmap_geo_us_counties,fill_color=colors,stroke_color=colors,fill_opacity=0.8)
#gini_layer = gmaps.geojson_layer(x,fill_color=colors,stroke_color=colors)
fig.add_layer(poverty_layer)
fig

In [ ]:
len(gmap_geo_us_counties['features'])

In [ ]:
len(listx)

In [ ]:
len(list_counties)

In [ ]:
avg_prod_min

In [ ]:
avg_prod_max

In [ ]:
list_prod

In [ ]:
list_counties